In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz
from sklearn.preprocessing import OrdinalEncoder

In [2]:
train_df = pd.read_csv("rossmann-store-sales/train.csv", low_memory=False)
test_df = pd.read_csv("rossmann-store-sales/test.csv", low_memory=False)
store_df = pd.read_csv("rossmann-store-sales/store.csv", low_memory=False)

In [3]:
train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [5]:
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Dataset analysis
### Level of data:
* train_df: Store-Date
* test_df: Store-Date
* store_df: Store

In [19]:
print("Number of records in train_df: ", len(train_df))
print("Number of records in test_df: ", len(test_df))
print("Number of records in store_df: ", len(store_df))

print("Number of unique stores: ", train_df['Store'].nunique())
print("Number of unique ids (test set): ", test_df['Id'].nunique())

Number of records in train_df:  1017209
Number of records in test_df:  41088
Number of records in store_df:  1115
Number of unique stores:  1115
Number of unique ids (test set):  41088


## Merge Dataset

In [20]:
train_data = pd.merge(train_df, store_df, how='inner', on='Store')
test_data = pd.merge(test_df, store_df, how='inner', on='Store')
print(len(train_data))
print(len(test_data))

1017209
41088


In [92]:
print(train_data.shape)
print(test_data.shape)

(1017209, 18)
(41088, 17)


In [83]:
my_report = sweetviz.analyze([train_data, "train"], target_feat="Sales")

                                             |      | [  0%]   00:00 -> (? left)

In [84]:
my_report.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


## Testing Entity Embedding model

In [85]:
import Cat2Emb as c2e
from sklearn.model_selection import train_test_split
import imp
imp.reload(c2e)

<module 'Cat2Emb' from '/Users/suvendukumarpati/Documents/Cat2Emb/Cat2Emb.py'>

In [65]:
X_train, y_train =train_data.drop(['Sales'],axis=1), train_data['Sales']
print(X_train.shape)
print(y_train.shape)

(1017209, 17)
(1017209,)


In [67]:
X_train['Store'] = X_train['Store'].astype('object')
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  object 
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Customers                  1017209 non-null  int64  
 4   Open                       1017209 non-null  int64  
 5   Promo                      1017209 non-null  int64  
 6   StateHoliday               1017209 non-null  object 
 7   SchoolHoliday              1017209 non-null  int64  
 8   StoreType                  1017209 non-null  object 
 9   Assortment                 1017209 non-null  object 
 10  CompetitionDistance        1014567 non-null  float64
 11  CompetitionOpenSinceMonth  693861 non-null   float64
 12  CompetitionOpenSinceYear   693861 non-null   float64
 13  Promo2      

In [33]:
var = ['Store', 'Customers', 'Open', 'StoreType']

In [38]:
X_train[var].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   Store      1017209 non-null  object
 1   Customers  1017209 non-null  int64 
 2   Open       1017209 non-null  int64 
 3   StoreType  1017209 non-null  object
dtypes: int64(2), object(2)
memory usage: 38.8+ MB


In [120]:
oe = OrdinalEncoder()
X_train_enc = oe.fit_transform(X_train[['Store','StoreType']])
X_train_enc

array([[   0.,    2.],
       [   0.,    2.],
       [   0.,    2.],
       ...,
       [1114.,    3.],
       [1114.,    3.],
       [1114.,    3.]])

In [127]:
oe.categories_

[array([1, 2, 3, ..., 1113, 1114, 1115], dtype=object),
 array(['a', 'b', 'c', 'd'], dtype=object)]

In [130]:
X_train['StoreType'].nunique()

4

In [133]:
np.unique(X_train['StoreType'])[0]

'a'

In [129]:
pd.DataFrame({np.unique(X_train['StoreType'])[i]:i for i in range(X_train['StoreType'].nunique())}, columns=['cat', 'cat_idx'])

,cat,cat_idx


In [137]:
{np.unique(X_train['Store'])[i]:i for i in range(X_train['Store'].nunique())}

KeyboardInterrupt: 

In [86]:
embed_model = c2e.EmbeddingGenerator()
embed_model.fit(X_train[var], y_train)

Categorical vars:  ['Store', 'StoreType']
Numerical vars:  ['Customers', 'Open']
Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_Store (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 input_StoreType (InputLaye  [(None, 1)]                  0         []                            
 r)                                                                                               
                                                                                                  
 embedding_20 (Embedding)    (None, 1, 50)                55750     ['input_Store[0][0]']         
                                                                                                  
 embedding

In [119]:
pd.DataFrame(embed_model.model.get_layer('embedding_21').get_weights()[0]).add_prefix('cat_')

,cat_0,cat_1
0,-3.808167,-3.713932
1,-6.652787,-7.029023
2,-3.247117,-3.192783
3,-1.146268,-1.114312


In [118]:
embed_model.model.get_layer('embedding_20')

TypeError: get_input_at() missing 1 required positional argument: 'node_index'